### Variable Threshold

In [4]:
import cv2
import os, glob
import torch.nn as nn
import torch
import numpy as np
from dataset import *
from model import *
from torch.nn.parallel import DataParallel
import tqdm
import torchattacks
from tqdm import trange
from torch.utils.data import TensorDataset, DataLoader
import torchvision.models as tm
from matplotlib import pyplot as plt
import torchvision
from skimage.feature import local_binary_pattern
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

##=============Config===================
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
device='cuda'


### original inference process ###
stmodel= tm.resnet50().to(device)
stmodel = DataParallel(stmodel)
stmodel.load_state_dict(torch.load('checkpoint2/final_hog.pt'))
val_ltp = A.Compose([A.Lambda(name='ltp', image=ltp_transform_train, p=1, always_apply=True)])


val_dataset = featDataset(0, "val.txt", root='/hdd3/ILSVRC/Data/imagenet/', mode='val', feat='ltp')
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=24, drop_last=False, 
                                         num_workers=16, sampler=None, shuffle=True)
    
def atk_func(atk):
#     atk = torchattacks.FGSM(stmodel, eps=16/255)
    radius = 1
    n_points = 8 * radius
    stmodel.eval()
    print('Start validating')
    t = trange(len(val_loader), desc='Average Val Accuracy', leave=True)
    val_acc, val_acc2 = [], []
    for step_val, (feat,X,y) in zip(t, val_loader):
        X = X.to(device)
        y = y.to(device)
        c=feat.to(device)
        ##======================

#         adv_images1 = atk(X/255.0, y)
#         adv_images = adv_images1.permute(0,2,3,1).cpu().numpy().astype(np.float)*255

#         for b in range(adv_images.shape[0]):
#             adv_images[b] = val_ltp(image=adv_images[b])['image']
#     #         for c in range(3):
#     #             adv_images[b,:,:,c] =local_binary_pattern(adv_images[b,:,:,c], n_points, radius, 'ror')

#     #     adv_images[np.isnan(adv_images)] = 0
#     #     adv_images = (adv_images / float(2**8) -0.5) * 2
#         adv_images = (adv_images*0.125 -0.5) * 2

#         adv_images = torch.Tensor(adv_images).permute(0,3,1,2).cuda()
        
        norm_feat = (feat +1) / 4096.0
        adv_images = atk(norm_feat, y) * 4096 -1
        
        
        outputs = stmodel(adv_images)
        outputs2 = stmodel(feat)
        _, pred = torch.max(outputs.data, 1)
        val_acc2.append( (pred == y).sum().item()/y.size(0) )
        _, pred2 = torch.max(outputs2.data, 1)
        val_acc.append( (pred2 == y).sum().item()/y.size(0) )

    #     plt.imshow(adv_images[0].permute(1,2,0).cpu().data)
    #     plt.show()
        t.set_description("Average Val Accuracy: %.5f/Robustness: %.5f" % (np.mean(np.array(val_acc)), np.mean(np.array(val_acc2))))
        t.refresh() # to show immediately the update
#         im1 = adv_images[0].permute(1,2,0).cpu().numpy() 
#         plt.imshow(im1)
#         plt.show()
#         im2 = norm_feat[0].permute(1,2,0).cpu().numpy()
#         plt.imshow(im2)
#         plt.show()
#         import pdb
#         pdb.set_trace()
#         break

atk_func(torchattacks.PGD(stmodel, eps=8/255, steps=10))

The # of images is: 50000 on val mode!
Start validating


Average Val Accuracy: 0.65297/Robustness: 0.56490:  60%|███████████████████████████████████████████████████████████████████████████████████████▌                                                         | 1259/2084 [14:37<09:34,  1.44it/s]


KeyboardInterrupt: 

### BPDA

In [2]:
import cv2
import os, glob
import torch.nn as nn
import torch
import numpy as np
from dataset import *
from model import *
from torch.nn.parallel import DataParallel
import tqdm
import torchattacks
from tqdm import trange
from torch.utils.data import TensorDataset, DataLoader
import torchvision.models as tm
from matplotlib import pyplot as plt
import torchvision
from skimage.feature import local_binary_pattern
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torch.nn.functional as F
##=============Config===================
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
device='cuda'

class BPDA(nn.Module):

    def __init__(self):
        super(BPDA, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(3, 16, 3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 3, 3, stride=1, padding=1)
        # an affine operation: y = Wx + b


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return x


### original inference process ###

val_ltp = A.Compose([A.Lambda(name='ltp', image=ltp_transform_train, p=1, always_apply=True)])


val_dataset = featDataset(0, "val.txt", root='/hdd3/ILSVRC/Data/imagenet/', mode='val', feat='ltp')
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=24, drop_last=False, 
                                         num_workers=16, sampler=None, shuffle=True)
    
def atk_func():
#     t = trange(len(val_loader), desc='Average Val Accuracy', leave=True) 
    bpda = BPDA().to(device)
    mse = torch.nn.MSELoss()
    opt = torch.optim.Adam(bpda.parameters(), lr=0.01, weight_decay=1e-6)
    scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=30, gamma=0.1)
    for ep in tqdm.tqdm(range(90)):
#         t = trange(len(val_loader), desc='Average Val Accuracy', leave=True) 
        for step_val, (feat,X,y) in enumerate(val_loader):
            X = X.to(device)
            y = y.to(device)
            c=feat.to(device)
#             import pdb
#             pdb.set_trace()
            opt.zero_grad()
            pred = bpda(X)
            loss = mse(pred, c)
            loss.backward()
            opt.step()
            scheduler.step()
        print(loss.item())
    
        
    
    
    print('Start validating')
   
    val_acc, val_acc2 = [], []
    
    stmodel= tm.resnet50().to(device)
    stmodel = DataParallel(stmodel)
    stmodel.load_state_dict(torch.load('checkpoint2/final_hog.pt'))
    stmodel.eval()
    
    stmodel2 = nn.Sequential(bpda, stmodel)
    atk = torchattacks.PGD(stmodel2, eps=8/255, steps=10)
    t = trange(len(val_loader), desc='Average Val Accuracy', leave=True) 
    for step_val, (feat,X,y) in zip(t, val_loader):
        X = X.to(device)
        y = y.to(device)
        c=feat.to(device)
        ##======================

#         adv_images1 = atk(X/255.0, y)
#         adv_images = adv_images1.permute(0,2,3,1).cpu().numpy().astype(np.float)*255

#         for b in range(adv_images.shape[0]):
#             adv_images[b] = val_ltp(image=adv_images[b])['image']
#     #         for c in range(3):
#     #             adv_images[b,:,:,c] =local_binary_pattern(adv_images[b,:,:,c], n_points, radius, 'ror')

#     #     adv_images[np.isnan(adv_images)] = 0
#     #     adv_images = (adv_images / float(2**8) -0.5) * 2
#         adv_images = (adv_images*0.125 -0.5) * 2

#         adv_images = torch.Tensor(adv_images).permute(0,3,1,2).cuda()
        
        norm_feat = (feat +1) / 2
        adv_images = atk(norm_feat, y) /2-1
        
        
        outputs = stmodel(adv_images)
        outputs2 = stmodel(feat)
        _, pred = torch.max(outputs.data, 1)
        val_acc2.append( (pred == y).sum().item()/y.size(0) )
        _, pred2 = torch.max(outputs2.data, 1)
        val_acc.append( (pred2 == y).sum().item()/y.size(0) )

    #     plt.imshow(adv_images[0].permute(1,2,0).cpu().data)
    #     plt.show()
        t.set_description("Average Val Accuracy: %.5f/Robustness: %.5f" % (np.mean(np.array(val_acc)), np.mean(np.array(val_acc2))))
        t.refresh() # to show immediately the update
#         im1 = adv_images[0].permute(1,2,0).cpu().numpy() 
#         plt.imshow(im1)
#         plt.show()
#         im2 = norm_feat[0].permute(1,2,0).cpu().numpy()
#         plt.imshow(im2)
#         plt.show()
#         import pdb
#         pdb.set_trace()
#         break

atk_func()

The # of images is: 50000 on val mode!


  1%|██▏                                                                                                                                                                                                   | 1/90 [01:53<2:47:47, 113.11s/it]

0.5944359302520752


  2%|████▍                                                                                                                                                                                                 | 2/90 [03:43<2:43:18, 111.35s/it]

0.633886456489563


...

 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊  | 89/90 [2:43:43<01:50, 110.59s/it]

0.6765177845954895


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [2:45:34<00:00, 110.38s/it]

0.6423500776290894
Start validating



Average Val Accuracy: 0.58911/Robustness: 0.47175: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2084/2084 [26:01<00:00,  1.33it/s]


In [7]:
import cv2
import os, glob
import torch.nn as nn
import torch
import numpy as np
from dataset import *
from model import *
from torch.nn.parallel import DataParallel
import tqdm
import torchattacks
from tqdm import trange
from torch.utils.data import TensorDataset, DataLoader
import torchvision.models as tm
from matplotlib import pyplot as plt
import torchvision
from skimage.feature import local_binary_pattern
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torch.nn.functional as F
##=============Config===================
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
device='cuda'

class BPDA(nn.Module):

    def __init__(self):
        super(BPDA, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(3, 64, 3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 32, 3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 3, 3, stride=1, padding=1)
        # an affine operation: y = Wx + b


    def forward(self, x1):
        x = F.leaky_relu(self.conv1(x1))
        x = F.leaky_relu(self.conv2(x))
        x = F.leaky_relu(self.conv3(x))
        return x


### original inference process ###

val_ltp = A.Compose([A.Lambda(name='ltp', image=ltp_transform_train, p=1, always_apply=True)])


val_dataset = featDataset(0, "val.txt", root='/hdd3/ILSVRC/Data/imagenet/', mode='val', feat='ltp')
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=24, drop_last=False, 
                                         num_workers=16, sampler=None, shuffle=True)
    
def atk_func():
    t = trange(90, desc='bpda train', leave=True) 
    bpda = BPDA().to(device)
    mse = torch.nn.MSELoss()
    opt = torch.optim.Adam(bpda.parameters(), lr=0.01, weight_decay=1e-6)
    scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=30, gamma=0.1)
    for ep in t:
#         t = trange(len(val_loader), desc='Average Val Accuracy', leave=True) 
        loss1=0
        for step_val, (feat,X,y) in enumerate(val_loader):
            X = X.to(device)
            y = y.to(device)
            c=feat.to(device)
#             import pdb
#             pdb.set_trace()
            opt.zero_grad()
            pred = bpda(X)
            loss = mse(pred, c)
            loss.backward()
            opt.step()
            scheduler.step()
            loss1+= loss.item()
            t.set_description("bpda loss: %.5f" % (loss1/(1+step_val)))
            t.refresh()
    
        
    
    
    print('Start validating')
   
    val_acc, val_acc2 = [], []
    
    stmodel= tm.resnet50().to(device)
    stmodel = DataParallel(stmodel)
    stmodel.load_state_dict(torch.load('checkpoint2/final_hog.pt'))
    stmodel.eval()
    
    stmodel2 = nn.Sequential(bpda, stmodel)
    atk = torchattacks.PGD(stmodel2, eps=8/255, steps=10)
    t = trange(len(val_loader), desc='Average Val Accuracy', leave=True) 
    for step_val, (feat,X,y) in zip(t, val_loader):
        X = X.to(device)
        y = y.to(device)
        c=feat.to(device)
        ##======================

#         adv_images1 = atk(X/255.0, y)
#         adv_images = adv_images1.permute(0,2,3,1).cpu().numpy().astype(np.float)*255

#         for b in range(adv_images.shape[0]):
#             adv_images[b] = val_ltp(image=adv_images[b])['image']
#     #         for c in range(3):
#     #             adv_images[b,:,:,c] =local_binary_pattern(adv_images[b,:,:,c], n_points, radius, 'ror')

#     #     adv_images[np.isnan(adv_images)] = 0
#     #     adv_images = (adv_images / float(2**8) -0.5) * 2
#         adv_images = (adv_images*0.125 -0.5) * 2

#         adv_images = torch.Tensor(adv_images).permute(0,3,1,2).cuda()
        
        norm_feat = (feat +1) / 2
        adv_images = atk(norm_feat, y) /2-1
        
        
        outputs = stmodel(adv_images)
        outputs2 = stmodel(feat)
        _, pred = torch.max(outputs.data, 1)
        val_acc2.append( (pred == y).sum().item()/y.size(0) )
        _, pred2 = torch.max(outputs2.data, 1)
        val_acc.append( (pred2 == y).sum().item()/y.size(0) )

    #     plt.imshow(adv_images[0].permute(1,2,0).cpu().data)
    #     plt.show()
        t.set_description("Average Val Accuracy: %.5f/Robustness: %.5f" % (np.mean(np.array(val_acc)), np.mean(np.array(val_acc2))))
        t.refresh() # to show immediately the update
#         im1 = adv_images[0].permute(1,2,0).cpu().numpy() 
#         plt.imshow(im1)
#         plt.show()
#         im2 = norm_feat[0].permute(1,2,0).cpu().numpy()
#         plt.imshow(im2)
#         plt.show()
#         import pdb
#         pdb.set_trace()
#         break

atk_func()

The # of images is: 50000 on val mode!


bpda loss: 0.43430: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [2:46:42<00:00, 111.14s/it]


Start validating


Average Val Accuracy: 0.59087/Robustness: 0.46181: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2084/2084 [33:04<00:00,  1.05it/s]


In [1]:
import cv2
import os, glob
import torch.nn as nn
import torch
import numpy as np
from dataset import *
from model import *
from torch.nn.parallel import DataParallel
import tqdm
import torchattacks
from tqdm import trange
from torch.utils.data import TensorDataset, DataLoader
import torchvision.models as tm
from matplotlib import pyplot as plt
import torchvision
from skimage.feature import local_binary_pattern
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torch.nn.functional as F

class _ResidualDenseBlock(nn.Module):
    """Achieves densely connected convolutional layers.
    `Densely Connected Convolutional Networks <https://arxiv.org/pdf/1608.06993v5.pdf>` paper.
    Args:
        channels (int): The number of channels in the input image.
        growth_channels (int): The number of channels that increase in each layer of convolution.
    """

    def __init__(self, channels: int, growth_channels: int) -> None:
        super(_ResidualDenseBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels + growth_channels * 0, growth_channels, (3, 3), (1, 1), (1, 1))
        self.conv2 = nn.Conv2d(channels + growth_channels * 1, growth_channels, (3, 3), (1, 1), (1, 1))
        self.conv3 = nn.Conv2d(channels + growth_channels * 2, growth_channels, (3, 3), (1, 1), (1, 1))
        self.conv4 = nn.Conv2d(channels + growth_channels * 3, growth_channels, (3, 3), (1, 1), (1, 1))
        self.conv5 = nn.Conv2d(channels + growth_channels * 4, channels, (3, 3), (1, 1), (1, 1))

        self.leaky_relu = nn.LeakyReLU(0.2, True)
        self.identity = nn.Identity()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out1 = self.leaky_relu(self.conv1(x))
        out2 = self.leaky_relu(self.conv2(torch.cat([x, out1], 1)))
        out3 = self.leaky_relu(self.conv3(torch.cat([x, out1, out2], 1)))
        out4 = self.leaky_relu(self.conv4(torch.cat([x, out1, out2, out3], 1)))
        out5 = self.identity(self.conv5(torch.cat([x, out1, out2, out3, out4], 1)))
        out = torch.mul(out5, 0.2)
        out = torch.add(out, identity)

        return out


class _ResidualResidualDenseBlock(nn.Module):
    """Multi-layer residual dense convolution block.
    Args:
        channels (int): The number of channels in the input image.
        growth_channels (int): The number of channels that increase in each layer of convolution.
    """

    def __init__(self, channels: int, growth_channels: int) -> None:
        super(_ResidualResidualDenseBlock, self).__init__()
        self.rdb1 = _ResidualDenseBlock(channels, growth_channels)
        self.rdb2 = _ResidualDenseBlock(channels, growth_channels)
        self.rdb3 = _ResidualDenseBlock(channels, growth_channels)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.rdb1(x)
        out = self.rdb2(out)
        out = self.rdb3(out)
        out = torch.mul(out, 0.2)
        out = torch.add(out, identity)

        return out
    

##=============Config===================
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
device='cuda'

class BPDA(nn.Module):

    def __init__(self):
        super(BPDA, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(3, 64, 3, stride=1, padding=1)
        self.conv2 = _ResidualResidualDenseBlock(64,16)
        self.conv3 = _ResidualResidualDenseBlock(64,16)
        self.conv4 = nn.Conv2d(64, 3, 3, stride=1, padding=1)
        # an affine operation: y = Wx + b


    


    def forward(self, x1):
        x = F.leaky_relu(self.conv1(x1))
        x = (self.conv2(x))
        x = (self.conv3(x))
        x = F.leaky_relu(self.conv4(x))
        return x


### original inference process ###

val_ltp = A.Compose([A.Lambda(name='ltp', image=ltp_transform_train, p=1, always_apply=True)])


val_dataset = featDataset(0, "val.txt", root='/hdd3/ILSVRC/Data/imagenet/', mode='val', feat='ltp')
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, drop_last=False, 
                                         num_workers=16, sampler=None, shuffle=True)
    
def atk_func():
    t = trange(90, desc='bpda train', leave=True) 
    bpda = BPDA().to(device)
    mse = torch.nn.MSELoss()
    opt = torch.optim.AdamW(bpda.parameters(), lr=0.001, weight_decay=1e-6)
    scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=30, gamma=0.1)
    for ep in t:
#         t = trange(len(val_loader), desc='Average Val Accuracy', leave=True) 
        loss1=0
        for step_val, (feat,X,y) in enumerate(val_loader):
            X = X.to(device)
            y = y.to(device)
            c=feat.to(device)
#             import pdb
#             pdb.set_trace()
            opt.zero_grad()
            pred = bpda(X)
            loss = mse(pred, c)
            loss.backward()
            opt.step()
            scheduler.step()
            loss1+= loss.item()
            t.set_description("bpda loss: %.5f" % (loss1/(1+step_val)))
            t.refresh()
    
        
    
    
    print('Start validating')
   
    val_acc, val_acc2 = [], []
    
    stmodel= tm.resnet50().to(device)
    stmodel = DataParallel(stmodel)
    stmodel.load_state_dict(torch.load('checkpoint2/final_hog.pt'))
    stmodel.eval()
    
    stmodel2 = nn.Sequential(bpda, stmodel)
    atk = torchattacks.PGD(stmodel2, eps=8/255, steps=10)
    t = trange(len(val_loader), desc='Average Val Accuracy', leave=True) 
    for step_val, (feat,X,y) in zip(t, val_loader):
        X = X.to(device)
        y = y.to(device)
        c=feat.to(device)
        ##======================

#         adv_images1 = atk(X/255.0, y)
#         adv_images = adv_images1.permute(0,2,3,1).cpu().numpy().astype(np.float)*255

#         for b in range(adv_images.shape[0]):
#             adv_images[b] = val_ltp(image=adv_images[b])['image']
#     #         for c in range(3):
#     #             adv_images[b,:,:,c] =local_binary_pattern(adv_images[b,:,:,c], n_points, radius, 'ror')

#     #     adv_images[np.isnan(adv_images)] = 0
#     #     adv_images = (adv_images / float(2**8) -0.5) * 2
#         adv_images = (adv_images*0.125 -0.5) * 2

#         adv_images = torch.Tensor(adv_images).permute(0,3,1,2).cuda()
        
        norm_feat = (feat +1) / 2
        adv_images = atk(norm_feat, y) /2-1
        
        
        outputs = stmodel(adv_images)
        outputs2 = stmodel(feat)
        _, pred = torch.max(outputs.data, 1)
        val_acc2.append( (pred == y).sum().item()/y.size(0) )
        _, pred2 = torch.max(outputs2.data, 1)
        val_acc.append( (pred2 == y).sum().item()/y.size(0) )

    #     plt.imshow(adv_images[0].permute(1,2,0).cpu().data)
    #     plt.show()
        t.set_description("Average Val Accuracy: %.5f/Robustness: %.5f" % (np.mean(np.array(val_acc)), np.mean(np.array(val_acc2))))
        t.refresh() # to show immediately the update
#         im1 = adv_images[0].permute(1,2,0).cpu().numpy() 
#         plt.imshow(im1)
#         plt.show()
#         im2 = norm_feat[0].permute(1,2,0).cpu().numpy()
#         plt.imshow(im2)
#         plt.show()
#         import pdb
#         pdb.set_trace()
#         break

atk_func()

The # of images is: 50000 on val mode!


bpda loss: 0.60954: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [30:20:19<00:00, 1213.55s/it]


Start validating


Average Val Accuracy: 0.57298/Robustness: 0.43510: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [3:40:51<00:00,  4.24s/it]
